In [ ]:
from thinkbayes2 import Pmf, Cdf, Suite, MakeJoint
import thinkplot

import pymc3 as pm
import numpy as np

from pymc3.math import exp, log

In [ ]:
def plotCdf(values, *args, **kwargs):
    thinkplot.Cdf(Cdf(values), *args, **kwargs)

# Problem Description
In the 2018 season of Major League Baseball...

We want to estimate which team is the best team, and ...

To accomplish this, we build an object-oriented PyMC model that predicts the expected number of runs scored by a given team in a game against another team. The model also includes mechanisms for representing properties of Baseball as a game, allowing it to build a Bayesian estimate of the mean score in a baseball game and the additional runs that a team is expected to get from having home field advantage.

In [ ]:
class Baseball(object):
    """
    A Baseball object contains properties about fundamental properties of 
    the sport of Baseball itself, represented with PyMC3 distributions.
    """
    
    def __init__(self, name='baseball'):
        self.name = name
        
        self.mean_score = pm.Normal(f'{name}.mean_score', mu=4.5, sd=1) # Based on Sam's guess
        self.home_field_advantage = pm.Exponential(f'{name}.home_field_advantage', lam=3)  # goals

The model we use for runs is a Poisson process with parameter dependent on the skills of both teams, as well as which team has home field advantage. In particular, we model the expected number of runs a given team will score in a game as 

$$k_{baseball} * \frac{e^{skill_{you}}}{e^{skill_{opponent}}} + home_field_boost$$

where

\begin{cases} 
      home_field_boost =  & x\leq 0 \\
      \frac{100-x}{100} & 0\leq x\leq 100 \\
      0 & 100\leq x 
   \end{cases}

In [ ]:
class Team(object):
    def __init__(self, name, baseball):
        self.name = name
        self.baseball = baseball
        
        self.quality = pm.Normal(f'{name}.quality', mu=0, sd=1)
        
    def model_game(self, opponent, homefield : bool, observed_runs=None, name=None):
        if not name:
            name = f'{self.name}_v_{opponent.name}'
            
        expected_runs = self.baseball.mean_score * (exp(self.quality) / exp(opponent.quality))
        
        if homefield:
            expected_runs += self.baseball.home_field_advantage
        
        pm.Poisson(name, mu=expected_runs, observed=observed_runs)


In [ ]:
with pm.Model() as model:
    bb = Baseball()
    
    teams = {0: Team('team0', bb), 1: Team('team1', bb)}
    
    teams[1].model_game(teams[0], True, 10)
    teams[0].model_game(teams[1], False, 4)

In [ ]:
# Inspect the prior distribution

with model:
    prior_trace = pm.sample_prior_predictive()

for k, values in sorted(prior_trace.items()):
    if k.endswith('__'):
        continue
    plotCdf(values, label=k)
    thinkplot.Show()

In [ ]:
# Infer the posterior distributions
with model:
    trace = pm.sample(1000)
    
pm.traceplot(trace);

In [ ]:
for k in sorted(trace.varnames):
    values = trace[k]
    if k.endswith('__'):
        continue
    plotCdf(prior_trace[k], color='grey', label='prior')
    plotCdf(values, label='posterior')
    thinkplot.Config(title=k)
    thinkplot.Show()

In [ ]:
plotCdf(prior_trace['team1.quality'] - prior_trace['team0.quality'], label='prior', color='grey')
plotCdf(trace['team1.quality'] - trace['team0.quality'], label='posterior')
thinkplot.Config(title='Quality differential')

print('{} chance that Team 1 is better'.format(np.mean((trace['team1.quality'] - trace['team0.quality'])>0)))